In [2]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Carga de documentos

In [3]:
documents = SimpleDirectoryReader("data").load_data()

## Construccion del indice

In [4]:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/104 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/115 [00:00<?, ?it/s]

### Configuracion del retriever

In [5]:
retriever = VectorIndexRetriever(index=index, similarity_top_k=4)

### Response synthesizer

In [6]:
respose_synth = get_response_synthesizer()

### Construir query engine

In [8]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=respose_synth,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
)

### realizar la consulta al RAG

In [9]:
response = query_engine.query("Cuanto dinero demandaba al estado la Renta Dignidad el 2021?")

In [10]:
response.response

'El beneficio de la Renta Dignidad demandaba alrededor de Bs4.240 millones al Estado en 2021.'

In [11]:
for node in response.source_nodes:
    print(f"Score: {node.score} \t Text: {node.text[:100]}")

Score: 0.8573658510856512 	 Text: Como parte de las políticas de protección social, desde 2008 el Estado entrega a la población de 60 
Score: 0.8142438508422444 	 Text: independientes del Sistema Integral de Pensiones. Esta transferencia se entregó a más de cuatro 
mil
Score: 0.8128220740220005 	 Text: Tanto el Plan Nacional de Desarrollo 2006-2011 como el PDES 2016-
2020 dieron continuidad a las polí
Score: 0.7933720797623506 	 Text: Asegurados en el sistema de pensiones 
En la última década, según los registros del sistema de pensi


### guardar indice en el disco

In [12]:
index.storage_context.persist(persist_dir="mi_indice")

In [13]:
from llama_index.core import StorageContext, load_index_from_storage
PERSIST_DIR = "mi_indice"

if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader("data").load_data()
    print("creando indice")
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    print("cargando indice")
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)


cargando indice
